# ChatBots in a Jupyter Notebook

Let's see how to use the ChatBot class to enable you to chat with Mistral inside a Jupyter notebook.

In [ ]:
from llamabot import ChatBot

code_tester = ChatBot(
    """
You are a Python quality assurance developer who delivers high quality unit tests for code.
You write tests using PyTest and not the built-in unittest library.
Write the tests using test functions and not using classes and class methods
Here is the code to write tests against:
""",
    session_name="code-tested",
    model_name="mistral/mistral-medium",
    stream_target="stdout",
)

In [ ]:
code_tester(
    '''
class ChatBot:
    """Chat Bot that is primed with a system prompt, accepts a human message.

    Automatic chat memory management happens.

    h/t Andrew Giessel/GPT4 for the idea.
    """

    def __init__(self, system_prompt, temperature=0.0, model_name="gpt-4"):
        """Initialize the ChatBot.

        :param system_prompt: The system prompt to use.
        :param temperature: The model temperature to use.
            See https://platform.openai.com/docs/api-reference/completions/create#completions/create-temperature
            for more information.
        :param model_name: The name of the OpenAI model to use.
        """
        self.model = ChatOpenAI(
            model_name=model_name,
            temperature=temperature,
            streaming=True,
            verbose=True,
            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        )
        self.chat_history = [
            SystemMessage(content="Always return Markdown-compatible text."),
            SystemMessage(content=system_prompt),
        ]

    def __call__(self, human_message) -> Response:
        """Call the ChatBot.

        :param human_message: The human message to use.
        :return: The response to the human message, primed by the system prompt.
        """
        self.chat_history.append(HumanMessage(content=human_message))
        response = self.model(self.chat_history)
        self.chat_history.append(response)
        return response
'''
)

Here are the tests for the ChatBot class using PyTest and test functions:
```python
import pytest
from chatbot import ChatBot, SystemMessage, HumanMessage
from openai_callback import ChatOpenAI

def test_chatbot_init():
    system_prompt = "You are a helpful assistant."
    chatbot = ChatBot(system_prompt)
    assert len(chatbot.chat_history) == 2
    assert isinstance(chatbot.chat_history[0], SystemMessage)
    assert isinstance(chatbot.chat_history[1], SystemMessage)
    assert chatbot.chat_history[0].content == "Always return Markdown-compatible text."
    assert chatbot.chat_history[1].content == system_prompt

def test_chatbot_call():
    system_prompt = "You are a helpful assistant."
    chatbot = ChatBot(system_prompt)
    human_message = "What is the weather like today?"
    response = chatbot(human_message)
    assert len(chatbot.chat_history) == 4
    assert isinstance(chatbot.chat_history[2], HumanMessage)
    assert isinstance(chatbot.chat_history[3], ChatOpenAI.Response)
 

AIMessage(content='', role='assistant')

As you can see, ChatBot keeps track of conversation memory/history automatically.
We can even access any item in the conversation by looking at the conversation history.

The `__repr__` of a chatbot will simply print out the entire history:

In [ ]:
code_tester

[Human]

class ChatBot:
    """Chat Bot that is primed with a system prompt, accepts a human message.

    Automatic chat memory management happens.

    h/t Andrew Giessel/GPT4 for the idea.
    """

    def __init__(self, system_prompt, temperature=0.0, model_name="gpt-4"):
        """Initialize the ChatBot.

        :param system_prompt: The system prompt to use.
        :param temperature: The model temperature to use.
            See https://platform.openai.com/docs/api-reference/completions/create#completions/create-temperature
            for more information.
        :param model_name: The name of the OpenAI model to use.
        """
        self.model = ChatOpenAI(
            model_name=model_name,
            temperature=temperature,
            streaming=True,
            verbose=True,
            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        )
        self.chat_history = [
            SystemMessage(content="Always return Markdown-compatible 

On the other hand, accessing the `.messages` attribute of the ChatBot will give you access to all of the messages inside the conversation.

In [ ]:
code_tester.messages

[HumanMessage(content='\nclass ChatBot:\n    """Chat Bot that is primed with a system prompt, accepts a human message.\n\n    Automatic chat memory management happens.\n\n    h/t Andrew Giessel/GPT4 for the idea.\n    """\n\n    def __init__(self, system_prompt, temperature=0.0, model_name="gpt-4"):\n        """Initialize the ChatBot.\n\n        :param system_prompt: The system prompt to use.\n        :param temperature: The model temperature to use.\n            See https://platform.openai.com/docs/api-reference/completions/create#completions/create-temperature\n            for more information.\n        :param model_name: The name of the OpenAI model to use.\n        """\n        self.model = ChatOpenAI(\n            model_name=model_name,\n            temperature=temperature,\n            streaming=True,\n            verbose=True,\n            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),\n        )\n        self.chat_history = [\n            SystemMessage(co

You can even access any arbitrary message.

In [ ]:
print(code_tester.messages[-1].content)

Here are some tests for the ChatBot class using PyTest:
```python
import pytest
from your_module import ChatBot, SystemMessage, HumanMessage

def test_init():
    system_prompt = "You are a helpful assistant."
    chatbot = ChatBot(system_prompt)
    assert len(chatbot.chat_history) == 2
    assert isinstance(chatbot.chat_history[0], SystemMessage)
    assert isinstance(chatbot.chat_history[1], SystemMessage)
    assert chatbot.chat_history[0].content == "Always return Markdown-compatible text."
    assert chatbot.chat_history[1].content == system_prompt

def test_call():
    system_prompt = "You are a helpful assistant."
    chatbot = ChatBot(system_prompt)
    human_message = "What's the weather like today?"
    response = chatbot(human_message)
    assert len(chatbot.chat_history) == 4
    assert isinstance(chatbot.chat_history[2], HumanMessage)
    assert isinstance(chatbot.chat_history[3], response.__class__)
    assert chatbot.chat_history[2].content == human_message

def test_re